## Squared Norm Regularization

Weight Decay is also called L2, squared norm regularization.

$$ \ell(w_1, w_2, b) = \frac{1}{n} \sum_{i=1}^n \frac{1}{2}\left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right)^2 $$

为例，其中$w_1, w_2$是权重参数，$b$是偏差参数，样本$i$的输入为$x_1^{(i)}, x_2^{(i)}$，标签为$y^{(i)}$，样本数为$n$。将权重参数用向量$\boldsymbol{w} = [w_1, w_2]$表示，带有$L_2$范数惩罚项的新损失函数为

$$\ell(w_1, w_2, b) + \frac{\lambda}{2n} |\boldsymbol{w}|^2,$$

其中超参数$\lambda > 0$。当权重参数均为0时，惩罚项最小。当$\lambda$较大时，惩罚项在损失函数中的比重较大，这通常会使学到的权重参数的元素较接近0。当$\lambda$设为0时，惩罚项完全不起作用。上式中$L_2$范数平方$|\boldsymbol{w}|^2$展开后得到$w_1^2 + w_2^2$。有了$L_2$范数惩罚项后，在小批量随机梯度下降中，我们将线性回归一节中权重$w_1$和$w_2$的迭代方式更改为

$$ \begin{aligned} 
w_1 &\leftarrow \left(1- \frac{\eta\lambda}{|\mathcal{B}|} \right)w_1 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}}x_1^{(i)} \left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right), \\
w_2 &\leftarrow \left(1- \frac{\eta\lambda}{|\mathcal{B}|} \right)w_2 - \frac{\eta}{|\mathcal{B}|} \sum_{i \in \mathcal{B}}x_2^{(i)} \left(x_1^{(i)} w_1 + x_2^{(i)} w_2 + b - y^{(i)}\right). 
\end{aligned} $$

可见，$L_2$范数正则化令权重$w_1$和$w_2$先自乘小于1的数，再减去不含惩罚项的梯度。因此，$L_2$范数正则化又叫权重衰减。权重衰减通过惩罚绝对值较大的模型参数为需要学习的模型增加了限制，这可能对过拟合有效。实际场景中，我们有时也在惩罚项中添加偏差元素的平方和。

I will try to explore some of the concepts.

Regularization means essentially "prior" of the weights, makeing distribution assumptions over the weights, which can have the attributes we are expecting, like, the weights are not too huge or too small, so that we can have a relatively smooth curve. Thus the overfitting can be reduced. 

正则化本质上对W的先验概率，即假设权重W符合某种分布，这种分布可以让W具有一些我们期望的特点，比如说都接近0，数值相差不大，这样拟合出的曲线就会相对平滑，从而降低了overfitting的可能性。

There are two kind of assumptions， L1 and L2

The first one, W lies under normal distribution. It's called L2 regularization.

MAP(Maximun a posteriori) = MLE(Maximum Likelihood Estimation) * Prior

$$ \begin{aligned} 
w = \underset{w}{\operatorname{argmax}}\prod_{i}^n\mathbf{N}(y_i|w\cdot x_i, \sigma^2)\mathbf{N}(w|0, 1/\lambda)
\end{aligned} $$

The second one, W lies under Laplace distribution. It's called L1 regularization.

$$ \begin{aligned} 
w = \underset{w}{\operatorname{argmax}}\prod_{i}^n\mathbf{N}(y_i|w\cdot x_i, \sigma^2)\mathbf{Laplace}(w)
\end{aligned} $$

Some resources about these concepts:

https://discuss.gluon.ai/t/topic/1793/4
https://www.zhihu.com/question/23536142
https://wiseodd.github.io/techblog/2017/01/01/mle-vs-map/




## High-demiensional Linear Regression

$$ y = 0.05 + \sum_{i = 1}^p 0.01x_i + \epsilon $$

In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append("..") 
import d2l_pytorch as d2l

n_train, n_test, num_inputs = 20, 100, 200
true_w, true_b = torch.ones(num_inputs, 1) * 0.01, 0.05

features = torch.randn((n_train + n_test, num_inputs))
labels = torch.matmul(features, true_w) + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)
train_features, test_features = features[:n_train, :], features[n_train:, :]
train_labels, test_labels = labels[:n_train], labels[n_train:]

In [2]:
def init_params():
    w = torch.randn((num_inputs, 1), requires_grad=True)
    b = torch.zeros(1, requires_grad=True)
    return [w, b]

In [3]:
def l2_penalty(w):
    return (w**2).sum() / 2

In [4]:
batch_size, num_epochs, lr = 1, 100, 0.003
net, loss = d2l.linreg, d2l.squared_loss

dataset = torch.utils.data.TensorDataset(train_features, train_labels)
train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)

def fit_and_plot(lambd):
    w, b = init_params()
    train_ls, test_ls = [], []
    for _ in range(num_epochs):
        for X, y in train_iter:
            # 添加了L2范数惩罚项
            l = loss(net(X, w, b), y) + lambd * l2_penalty(w)
            l = l.sum()
            
            if w.grad is not None:
                w.grad.data.zero_()
                b.grad.data.zero_()
            l.backward()
            d2l.sgd([w, b], lr, batch_size)
        train_ls.append(loss(net(train_features, w, b), train_labels).mean().item())
        test_ls.append(loss(net(test_features, w, b), test_labels).mean().item())
    d2l.semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'loss',
                 range(1, num_epochs + 1), test_ls, ['train', 'test'])
    print('L2 norm of w:', w.norm().item())